# Nearest Neighbor for Spine Injury Classification

In this notebook, we use **nearest neighbor classification** to classify back injuries for patients in a hospital, based on measurements of the shape and orientation of their pelvis and spine.

The data set contains information from **310** patients. For each patient, there are: six measurements (the x) and a label (the y). The label has **3** possible values, `’NO’` (normal), `’DH’` (herniated disk), or `’SL’` (spondilolysthesis). 

# 1. Setup notebook

We import all necessary packages for the data analysis. Notice that we do **NOT** import any of the `sklearn` packages. This is because we want to implement a nearest neighbor classifier **manually** to understand how this algorithm works.


In [1]:
import numpy as np

We now load the dataset. We divide the data into a training set of 248 patients and a separate test set of 62 patients. The following arrays are created:

* **`trainx`** : The training data's features, one point per row.
* **`trainy`** : The training data's labels.
* **`testx`** : The test data's features, one point per row.
* **`testy`** : The test data's labels.

We will use the training set (`trainx` and `trainy`), with nearest neighbor classification, to predict labels for the test data (`testx`). We will then compare these predictions with the correct labels, `testy`.

Notice that we code the three labels as `0. = ’NO’, 1. = ’DH’, 2. = ’SL’`.

In [2]:
# Load data set and code labels as 0 = ’NO’, 1 = ’DH’, 2 = ’SL’
labels = [b'NO', b'DH', b'SL']
fname = 'column_3C.dat'
data = np.loadtxt(fname, converters={6: lambda s: labels.index(s)} )

# Separate features from labels
x = data[:,0:6]
y = data[:,6]

# Divide into training and test set
training_indices = list(range(0,20)) + list(range(40,188)) + list(range(230,310))
test_indices = list(range(20,40)) + list(range(188,230))

trainx = x[training_indices,:]
trainy = y[training_indices]
testx = x[test_indices,:]
testy = y[test_indices]
print(testy[0:10])

print('size of trainx: ',trainx.shape)
print('size of trainy: ',trainy.shape)
print('size of testx: ',testx.shape)
print('size of testy: ',testy.shape)

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
size of trainx:  (248, 6)
size of trainy:  (248,)
size of testx:  (62, 6)
size of testy:  (62,)


## 2. Nearest neighbor classification with $L_2$ distance

We now build a nearest neighbor (NN) classifier based on $L_2$ (*Euclidean*) distance,

$$L_2 = \sqrt{\sum_{i=1}^m (x_i - z_i)^2}, $$

where $x,z \in \mathbb{R}^m$.

<font color="magenta">**Goal:**</font> Write a function, **NN_L2**, which takes as input the training data (`trainx` and `trainy`) and the test points (`testx`) and predicts labels for these test points using 1-NN classification. These labels should be returned in a `numpy` array with one entry per test point. For **NN_L2**, the L2 norm should be used as the distance metric.

In [8]:
## Computes squared Euclidean distance between two vectors.
def squared_dist(x,y):
    return np.sqrt(np.sum(np.square(x-y)))

def NN_L2(trainx, trainy, testx):
    # inputs: trainx, trainy, testx 
    # output: an np.array of the predicted values for testy 
    
    # Find the nearest neighbor for each train example
    testy_L2 = np.zeros(testx.shape[0])
    for j in range(testx.shape[0]):
        distances = [squared_dist(testx[j,:],trainx[i,:]) for i in range(len(trainy))]
        index = np.argmin(distances)
        testy_L2[j] = trainy[index] 
    return testy_L2
    

<font  style="color:blue"> **Code**</font>
```python
# test function 
testy_L2 = NN_L2(trainx, trainy, testx)
print( type( testy_L2) )
print( len(testy_L2) )
print( testy_L2[40:50] )
```

<font  style="color:magenta"> **Output**</font>
```
<class 'numpy.ndarray'>
62
[ 2.  2.  1.  0.  0.  0.  0.  0.  0.  0.]
```



In [9]:
testy_L2 = NN_L2(trainx, trainy, testx)
print(type(testy_L2))
print(len(testy_L2))
print( testy_L2[40:50] )

<class 'numpy.ndarray'>
62
[2. 2. 1. 0. 0. 0. 0. 0. 0. 0.]


After you are done, run the cell below to check our function. If an error is triggered, you should go back and revise our function.

In [10]:
testy_L2 = NN_L2(trainx, trainy, testx)

assert( type( testy_L2).__name__ == 'ndarray' )
assert( len(testy_L2) == 62 ) 
assert( np.all( testy_L2[50:60] == [ 0.,  0.,  0.,  0.,  2.,  0.,  2.,  0.,  0.,  0.] )  )
assert( np.all( testy_L2[0:10] == [ 0.,  0.,  0.,  1.,  1.,  0.,  1.,  0.,  0.,  1.] ) )

# 3. Nearest neighbor classification with L1 distance

We now compute nearest neighbors using the L1 distance (sometimes called *Manhattan Distance*),

$$ L_1 = \sum_{i=1}^m |x_i - z_i|, $$
where $x,z$ are row vectors in trainx.

<font color="magenta">**Task:**</font> Write a function, **NN_L1**, which again takes as input the arrays `trainx`, `trainy`, and `testx`, and predicts labels for the test points using 1-nearest neighbor classification. For **NN_L1**, the L1 distance metric should be used. As before, the predicted labels should be returned in a `numpy` array with one entry per test point.

Notice that **NN_L1** and **NN_L2** may well produce different predictions on the test set.


<font  style="color:blue"> **Code**</font>
```python
# test function 
testy_L2 = NN_L2(trainx, trainy, testx)
testy_L1 = NN_L1(trainx, trainy, testx)

print( type( testy_L1) )
print( len(testy_L1) )
print( testy_L1[40:50] )
print( all(testy_L1 == testy_L2) )
```

<font  style="color:magenta"> **Output**</font>
```
<class 'numpy.ndarray'>
62
[ 2.  2.  0.  0.  0.  0.  0.  0.  0.  0.]
False
```


In [13]:
## Computes L1 distance between two vectors.
def L1_dist(x,y):
    return np.sum(np.absolute(x-y))

def NN_L1(trainx, trainy, testx):
    # inputs: trainx, trainy, testx <-- as defined above
    # output: an np.array of the predicted values for testy 
    
    # Find the nearest neighbor for each train example
    testy_L1 = np.zeros(testx.shape[0])
    for j in range(testx.shape[0]):
        distances = [L1_dist(testx[j,:],trainx[i,:]) for i in range(len(trainy))]
        index = np.argmin(distances)
        testy_L1[j] = trainy[index] 
    return testy_L1

In [14]:
testy_L1 = NN_L1(trainx, trainy, testx)
print( type( testy_L1) )
print( len(testy_L1) )
print( testy_L1[40:50] )
print( np.all(testy_L1 == testy_L2) )

<class 'numpy.ndarray'>
62
[2. 2. 0. 0. 0. 0. 0. 0. 0. 0.]
False


Again, use the following cell to check your code.

In [16]:
testy_L1 = NN_L1(trainx, trainy, testx)
testy_L2 = NN_L2(trainx, trainy, testx)

assert( type( testy_L1).__name__ == 'ndarray' )
assert( len(testy_L1) == 62 ) 
assert( not all(testy_L1 == testy_L2) )
assert( np.all(testy_L1[50:60] == [ 0.,  2.,  1.,  0.,  2.,  0.,  0.,  0.,  0.,  0.] ) )
assert( np.all( testy_L1[0:10] == [ 0.,  0.,  0.,  0.,  1.,  0.,  1.,  0.,  0.,  1.]) )

# 4. Test errors and the confusion matrix

Let's see if the $L_1$ and $L_2$ distance functions yield different error rates for nearest neighbor classification of the test data.

In [17]:
def error_rate(testy, testy_fit):
    return float(sum(testy!=testy_fit))/len(testy) 


In [18]:
print(testy[0:10])
print(testy_L1[0:10])
print(testy_L2[0:10])
print("Error rate of NN_L1: ", error_rate(testy,testy_L1) )
print("Error rate of NN_L2: ", error_rate(testy,testy_L2) )

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[0. 0. 0. 0. 1. 0. 1. 0. 0. 1.]
[0. 0. 0. 1. 1. 0. 1. 0. 0. 1.]
Error rate of NN_L1:  0.22580645161290322
Error rate of NN_L2:  0.20967741935483872


Hence, $L_1$ and $L_2$ yield different error rates.

We will now look a bit more deeply into the specific types of errors made by nearest neighbor classification, by constructing the <font color="magenta">*confusion matrix*</font>.

Since there are three labels, the confusion matrix is a 3x3 matrix that shows the number of misclassifications for each label. For example, the entry at row DH, column SL, contains the number of test points whose correct label was DH but which were classified as SL.

<img style="width:200px" src="Figure/confusion_matrix.png">




Write a function, **confusion**, which takes as input the true labels for the test set (that is, `testy`) as well as the predicted labels and returns the confusion matrix. The confusion matrix should be a `np.array` of shape `(3,3)` . 

In [29]:
def confusion(testy,testy_fit):
    # inputs: testy: the correct labels, testy_fit: the fitted NN labels 
    # output: a 3x3 np.array representing the confusion matrix as above
    
    result = np.zeros((3,3))
    for i in range(len(testy)):
        if testy[i]== testy_fit[i]: continue
        row = int(testy[i])
        col = int(testy_fit[i])
        result[row, col] += 1.0
    return result

# Take a look at the output
L2_neo = confusion(testy,testy_L2)
print(type(L2_neo))
print(L2_neo.shape)
print('confusion matrix for L2:')
print(L2_neo)

L1_neo = confusion(testy,testy_L1)
print('confusion matrix for L1:')
print(L1_neo)

# Count how many test points yield different predictions based on L1 and L2 distance
print('number of test points predicted differently by L1 and L2 distance:',sum(testy_L1 != testy_L2))

<class 'numpy.ndarray'>
(3, 3)
confusion matrix for L2:
[[ 0.  1.  2.]
 [10.  0.  0.]
 [ 0.  0.  0.]]
confusion matrix for L1:
[[ 0.  2.  2.]
 [10.  0.  0.]
 [ 0.  0.  0.]]
number of test points predicted differently by L1 and L2 distance: 7


In [25]:
# Test Function

L1_neo = confusion(testy, testy_L1) 
assert( type(L1_neo).__name__ == 'ndarray' )
assert( L1_neo.shape == (3,3) )
assert( np.all(L1_neo == [[ 0.,  2.,  2.],[ 10.,  0.,  0.],[ 0.,  0.,  0.]]) )
L2_neo = confusion(testy, testy_L2)  
assert( np.all(L2_neo == [[ 0.,  1.,  2.],[ 10.,  0.,  0.],[ 0.,  0.,  0.]]) )

# 5. Discussion

* In the test set, the class DH was **most frequently** misclassified by the L1-based nearest neighbor classifier.
* In the test set, the class SL was **never** misclassified by the L2-based and L1-based nearest neighbor classifier.
* There is 7 of the test point leading to *different* prediction based on L1 and L2 distance.